# Brain Wave Program

In [ ]:
train_data = pd.read_csv('/content/train_data.csv')
train_data

In [ ]:
train_data[['label', 'C3_Delta', 'C3_Theta', 'C3_Alpha', 'C3_Beta', 'C3_Gamma', 'C4_Delta', 'C4_Theta', 'C4_Alpha', 'C4_Beta', 'C4_Gamma']]

In [ ]:
X = train_data[['label', 'C3_Delta', 'C3_Theta', 'C3_Alpha', 'C3_Beta', 'C3_Gamma', 'C4_Delta', 'C4_Theta', 'C4_Alpha', 'C4_Beta', 'C4_Gamma']]

y = train_data["label"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
import optuna
import lightgbm as lgb
import xgbm as xgb
import catboost as cb
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import f1_score

def objective(trial):
    # Define hyperparameter search spaces
    xgb_params = {
        'max_depth': trial.suggest_int('xgb_max_depth', 2, 10),
        'n_estimators': trial.suggest_int('xgb_n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.3),
    }
    nn_params = {
        'lr': trial.suggest_float('nn_lr', 1e-5, 1e-2, log=True),
        'wd': trial.suggest_float('nn_wd', 1e-5, 1e-2, log=True),
    }
    lgb_params = {
        'max_depth': trial.suggest_int('lgb_max_depth', 2, 10),
        'num_leaves': trial.suggest_int('lgb_num_leaves', 10, 100),
        'learning_rate': trial.suggest_float('lgb_learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('lgb_n_estimators', 100, 1000),
    }
    cb_params = {
        'depth': trial.suggest_int('cb_depth', 2, 10),
        'iterations': trial.suggest_int('cb_iterations', 100, 1000),
        'learning_rate': trial.suggest_float('cb_learning_rate', 0.01, 0.3),
    }
    rf_params = {
        'n_estimators': trial.suggest_int('rf_n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('rf_max_depth', 2, 10),
    }

    # Create base models
    xgb_model = xgb.XGBClassifier(**xgb_params, random_state=42)
    nn_model = NeuralNetFastAI(**nn_params, random_state=42)
    lgb_model = lgb.LGBMClassifier(**lgb_params, random_state=42)
    cb_model = cb.CatBoostClassifier(**cb_params, random_state=42)
    rf_model = RandomForestClassifier(**rf_params, random_state=42)

    # Create voting classifier with given weights
    voting_clf = VotingClassifier(estimators=[
        ('XGBoost', xgb_model, 0.5),
        ('NeuralNetFastAI', nn_model, 0.2),
        ('LightGBM', lgb_model, 0.1),
        ('RandomForestEntr', rf_model, 0.1),
        ('CatBoost', cb_model, 0.1)
    ], voting='soft')

    # Fit and evaluate
    voting_clf.fit(X_train, y_train)
    y_pred = voting_clf.predict(X_val)
    f1 = f1_score(y_val, y_pred)
    return f1

# Assuming you have your data loaded and split into X_train, y_train, X_val, y_val

study = optuna.create_study(
    direction="maximize", study_name="voting-ensemble", storage="sqlite:///voting.db", load_if_exists=True)
study.optimize(objective, n_trials=50)

best_params = study.best_trial.params

print('Best parameters:', study.best_params)
print('Best score:', study.best_value)

xgb_model = xgb.XGBClassifier(
    **{k[4:]: v for k, v in best_params.items() if k.startswith('xgb_')})

nn_model = NeuralNetFastAI(
    **{k[3:]: v for k, v in best_params.items() if k.startswith('nn_')})

lgb_model = lgb.LGBMClassifier(
    **{k[4:]: v for k, v in best_params.items() if k.startswith('lgb_')})

cb_model = cb.CatBoostClassifier(
    **{k[3:]: v for k, v in best_params.items() if k.startswith('cb_')})

rf_model = RandomForestClassifier(
    **{k[3:]: v for k, v in best_params.items() if k.startswith('rf_')})

voting_clf = VotingClassifier(estimators=[
    ('XGBoost', xgb_model, 0.5),
    ('NeuralNetFastAI', nn_model, 0.2),
    ('LightGBM', lgb_model, 0.1),
    ('RandomForestEntr', rf_model, 0.1),
    ('CatBoost', cb_model, 0.1)
], voting='soft')

voting_clf.fit(X_train, y_train)

import joblib

joblib.dump(voting_clf, "voting-ensemble.pkl")